In [ ]:
import json

from time import sleep
from random import randint

import pandas as pd

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2

In [ ]:
from web_crawler.scraper_migros import MigrosScraper
from web_crawler.scraper_cora import CoraScraper
from web_crawler.scraper_coop import CoopScraper

In [ ]:
products = pd.read_pickle("./products_pd.pickle")

In [ ]:
products.loc[products["stores"].apply({"Coop"}.issubset)]

In [ ]:
products.product_name = products.product_name.fillna("")
products.brands = products.brands.fillna("")

migros = MigrosScraper()
cora = CoraScraper()

with open('scraped_products.json', mode='w', encoding='utf-8') as f:

    for i, row in products.iterrows():
        if "Migros" in row.stores:
            query = row['product_name']
            print(query.title())
            try:
                product_dict = migros.search(query)
                product_dict['code'] = row.name
                product_dict['product_name'] = row.product_name
            except Exception as err:
                print(err)
                continue
                
            f.write(json.dumps(product_dict)) # use `json.loads` to do the reverse
            
        elif "Cora" in row.stores:
            query = row['product_name']
            print(query.title())
            try:
                product_dict = cora.search(query)
                product_dict['code'] = row.name
                product_dict['product_name'] = row.product_name
            except Exception as err:
                print(err)
                continue
                
            f.write(json.dumps(product_dict) + ',') # use `json.loads` to do the reverse
            
            # We don't wanna get banned from the server
            sleep(randint(1,10))
            

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
BASE_URL = "https://search.migros.ch/de/q:"

driver = webdriver.Firefox(executable_path='/home/kingkolibri/Programs/geckodriver')
driver.implicitly_wait(2)

In [ ]:
query = "Chocolate Chip Shortbread"

driver.get(BASE_URL + query )




In [ ]:
# Find object on search results page
python_button = driver.find_element_by_css_selector(
    '.mui-list-products-wide > li:nth-child(1) > a:nth-child(1)')  # First article in grid view
python_button.click()  # click link

# Hand the page source to Beautiful Soup
soup = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
''.join(soup.select_one('.sidebar-product-name').text).strip()

In [ ]:
soup.select_one('.current-price').text

In [ ]:
''.join(soup.select_one('p.sidebar-subtext').text).strip()

In [ ]:
soup.select_one('.mui-breadcrumb').text.split()

In [ ]:
element = soup.select_one('#title')

In [ ]:
'-'.join([i.text for i in soup.select_one('#title').findAll('span')[:-2]])